In [ ]:
# Ejecutar esta celda solo si la celda de abajo tuvo problemas
# Recargar página luego de ejecutar esta celda por primera vez
!pip install "ipywidgets>=7.2"
!pip install sklearn
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install bqplot
!jupyter nbextension enable --py --sys-prefix bqplot

In [ ]:
import pandas as pd
import numpy as np
import bqplot.pyplot as plt
import tensorflow as tf
from IPython.display import display
import ipywidgets as widgets

In [ ]:
# Importar las funciones de los modelos
from TimeSeriesPrediction import fixed_partitioning_predict
from LinearRegression import linear_regression_predict
from DNN import load_model, dnn_predict, dnn_predict_by_year

# Importar interface de modelos
from Evaluation import Model, TestResult

# Crear modelos

model_dict = {
    'Fix. part. time series' : Model(fixed_partitioning_predict),
    'Fix. part. linear regression' : Model(linear_regression_predict),
    'Deep Neural Networks' : Model(dnn_predict, args = dict(cached_model = load_model('default')))
}

# Crear etiquetas
label_dict = {
    'Fix. part. time series' : 'Fixed partitioning time series',
    'Fix. part. linear regression' : 'Fixed partitioning linear regression',
    'Deep Neural Networks' : 'Redes neuronales', 
    'Primarias públicas y privadas' : 'PrimariasCompletas',
    'Primarias públicas' : 'PrimariasPublicas',
    'Primarias privadas' : 'PrimariasPrivadas'
}

In [ ]:
# Seleccionar modelo de predicción
model_radio_button = widgets.RadioButtons(
    options=['Fix. part. time series', 'Fix. part. linear regression', 'Deep Neural Networks'],
    description='Modelo',
    disabled=False
)

display(model_radio_button)

In [ ]:
# Seleccionar años a predecir
prediction_size_int_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=5,
    step=1,
    description='Años',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(prediction_size_int_slider)

In [ ]:
# Seleccionar conjunto de datos
set_name_radio_button = widgets.RadioButtons(
    options=['Primarias públicas y privadas', 'Primarias públicas', 'Primarias privadas'],
    description='Conjunto',
    disabled=False
)

display(set_name_radio_button)

In [ ]:
# Calcular resultados de la prueba
model = model_dict[model_radio_button.value]
result = model.test_set(label_dict[set_name_radio_button.value], prediction_size_int_slider.value)

m = max(np.amax(result.Y_hat), np.amax(result.Y))
print("Mean absolute percentage error: %f" % (result.mape))
print("Mean absolute error: %f" % (result.mae))

fig = plt.figure(title = 'Predicción de matrícula escolar en %s utilizando %s' % (set_name_radio_button.value,model_radio_button.value))
plt.plot(x = np.array([0, m]), y = np.array([0, m]))
plt.scatter(result.Y, result.Y_hat, stroke='black')
fig

# Predicción de matrícula en una sola escuela

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual

out = widgets.Output()

@interact(cct = 'Introduce cct', 
          anios = widgets.IntSlider(min=1, max=5, step=1, value=3, description = 'Años'), 
          modelo = widgets.RadioButtons(
              options=['Fix. part. time series', 
                       'Fix. part. linear regression', 
                       'Deep Neural Networks'],
              description='Modelo',
              disabled=False
))
def prediccion(cct, anios, modelo) :
    global model_dict
    model = model_dict[modelo]
    
    dataset = pd.read_csv('PrimariasCompletas.csv')
    unique_index = pd.Index(list(dataset['cct']))
    if cct in unique_index :
        index = unique_index.get_loc(cct)
        print('Se encontró el cct')
    else :
        print('No se encontró el cct')
        return
    
    # Quitar el cct
    row = np.array(dataset.loc[index][1:])l
    
    # Separamos los últimos 5 años
    X = row[: -model.TEST_SIZE]

    if anios == model.TEST_SIZE :
        Y = row[-model.TEST_SIZE :]
    else :
        Y = row[-model.TEST_SIZE : -(model.TEST_SIZE - anios)]
    prediccion = model.predict(X, anios)
    
    list_X = list(X)
    list_Y = list(Y)
    list_predict = list(prediccion)
    
    with out:
        out.clear_output(True)
        
        axes_options = {'x': {'label': 'Año'},'y': {'label': 'Alumnos'}}
        
        fig = plt.figure(title = 'Predicción de matrícula escolar en la escuela %s utilizando %s' % (cct, modelo))
        plt.plot(x = [1998 + i for i in range(len(X))], y = list_X, colors = ['red'], axes_options=axes_options)
        plt.plot(x = [1997 + len(X) + i  for i in range(len(Y) + 1)], y = (list_X + list_Y)[-(anios + 1):])
        plt.plot(x = [1997 + len(X) + i  for i in range(len(Y) + 1)], y = (list_X + list_predict)[-(anios + 1):], colors = ['green'])
        
        plt.show()
out